In [5]:
import pandas as pd
import glob
import os

In [3]:
binsizes = ['10kb', '50kb', '500kb', '1000kb']
base_path = os.getcwd()


In [4]:
for binsize in binsizes:
    files = glob.glob(os.path.join(base_path, f'*_{binsize}_overlaps.tsv'))
    rdv = []
    mms = []

    for file in files:
        sample = os.path.basename(file).replace(f'_{binsize}_overlaps.tsv', '')

        # Rename MM → MMS universally
        sample = sample.replace('MM', 'MMS')

        # Read file
        df = pd.read_csv(file, sep='\t', header=None)

        df.columns = ['chr', 'start', 'end', 'event', 'gene_chr', 'gene_start', 'gene_end', 'gene_name']

        # Group genes
        grouped = df.groupby(['chr', 'event'])['gene_name'].unique().reset_index()
        grouped['sample'] = sample
        grouped = grouped[['sample', 'chr', 'event', 'gene_name']]

        # Assign to group
        if 'RDV' in sample:
            rdv.append(grouped)
        elif 'MMS' in sample:
            mms.append(grouped)

    if rdv:
        pd.concat(rdv, ignore_index=True).to_csv(f'RDV_{binsize}_overlaps.csv', index=False)
    if mms:
        pd.concat(mms, ignore_index=True).to_csv(f'MMS_{binsize}_overlaps.csv', index=False)